In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Equations ###
Our first task is to simulate the flow of heat in a solid.

The heat flow equation is:

$$ \frac{\partial u}{\partial t} = \frac{K}{C\rho}\nabla^2 u $$

$$ = \frac{K}{C\rho} \Big(\frac{\partial^2 u}{\partial x^2}
+ \frac{\partial^2 u}{\partial y^2}
+ \frac{\partial^2 u}{\partial z^2}\Big) $$

Where $u$ is the temperature, $K$ is the thermal conductivity, $C$ is the specific heat, and $\rho$ is the density. For brevity, I will define $A = \frac{K}{C\rho}$.

We solve this using a timestep algorithm.

#### Discrete Version ####
At a given point $u_i(t_{i})$, the spatial derivatives can be expressed in discrete form:
$$ \frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1} - 2u_i + u_{i-1}}{\Delta x^2} $$

And the time derivative can be expressed as:
$$ \frac{\partial u}{\partial t} = \frac{u(t_{n+1}) - u(t_n)}{\Delta t} $$

Given a particular temperature distribution, we may find the next iteration using the following time step equation:

$$ u_{i,j,k}(t_{n+1}) = u_{i,j,k}(t_n)
 + A\Big(
         \frac{u_{i+1,j,k} - 2u_{i,j,k} + u_{i-1,j,k}}{\Delta x^2}
         + \frac{u_{i,j+1,k} - 2u_{i,j,k} + u_{i,j-1,k}}{\Delta y^2}
         + \frac{u_{i,j,k+1} - 2u_{i,j,k} + u_{i,j,k-1}}{\Delta z^2}
    \Big) \Delta t $$

If we take the same spatial step size for each dimension, we may simplify the expression in parentheses:
$$ \Big(u_{i+1,j,k} + u_{i,j+1,k} + u_{i,j,k+1} + u_{i-1,j,k} + u_{i,j-1,k} + u_{i,j,k-1} - 6u_{i,j,k}\Big)
\cdot\frac{1}{\Delta x^2}
$$
$$
= \frac{1}{\Delta x^2} \bigg(\sum_{adjacent} u(t_n) - 6u_{i,j,k}(t_n)\bigg)
$$

Thus, we have the following equation for determining the next step in the simulation:
$$ u_{i,j,k}(t_{n+1}) = u_{i,j,k}(t_n)
+ A\frac{\Delta t}{\Delta x^2} \bigg(\sum_{adjacent} u(t_n) - 6u_{i,j,k}(t_n)\bigg)
$$

To handle an insulated boundary, we introduce fictional points such that the derivative will always be zero. As a simplified case, we assume that the non-insulated boundaries are in perfect thermal contact with a heat sink at a constant temperature.

#### Stability ####

Note that the stability of this function depends on the factor $A \Delta t / \Delta x^2$ being small. For a given material and a given spatial resolution, this means that the time division should satisfy $\Delta t < A \Delta x^2$.


## Python Code ##

### First step: Boundary Conditions ###

First I write a function that updates the fictional points. For the insulated boundaries, the fictional points are just outside the boundary and have the same temperature as the boundary.

For non-insulated boundaries, the points are held at a fixed temperature.

In [ ]:
np.ones(4) * 3


In [ ]:
def updateBoundary(T, T_ext, btype=(1,0,0,0,0,0)):
    """
    Updates the fictional points for the thermal conductor using
    a set of boundary conditions.
    
    The NumPy array is modified in-place.
    
    Arguments:
    ---------
    - T: a 3d array of temperatures within the solid. The shape of the
         array must be (h+1, w+2, d+2) to include the fictional points
         on 5 of the 6 surfaces.
         
    - T_ext: Temperature of the external environment. This may be either
        a scalar or an array. If it is an array, then the external
        temperatures at the 6 surfaces may be specified separately.
        The surfaces must be listed in the order (x0, x1, y0, y1, z0, z1)
        where x0 is the surface with the first index equal to zero, x1 is the
        surface opposite x0, and likewise for the other 4 surfaces.
    
    - btype: Array-like with the boundary types for the 6 surfaces.
        A value of 1 represents perfectly conducting and 0 represents
        perfectly insulating. Any value other than 0 or 1 will yield
        a linear combination of the two types.
        The surfaces are listed in the same order described above.
        
    Returns:
    ------
    Nothing
    
    """
    # Convert T_ext into an array. If it is already an array,
    # no change will occur.
    T_ext = np.ones(6)*T_ext
    
    # For a thermally conductive surface, the fictional points
    # have the temperature of the surroundings. For an insulating
    # surface, the fictional points have the same temperature as the
    # real points that they are in contact with.
    T[0,:,:] = T_ext[0]*btype[0] + T[1,:,:]*(1-btype[0])
    T[-1,:,:] = T_ext[1]*btype[1] + T[-2,:,:]*(1-btype[1])
    T[:,0,:] = T_ext[2]*btype[2] + T[:,1,:]*(1-btype[2])
    T[:,-1,:] = T_ext[3]*btype[3] + T[:,-2,:]*(1-btype[3])
    T[:,:,0] = T_ext[4]*btype[4] + T[:,:,1]*(1-btype[4])
    T[:,:,-1] = T_ext[5]*btype[5] + T[:,:,-2]*(1-btype[5])
    

### Second Step: Simulation ###

The next function simulates the flow of heat in a solid.

In [ ]:
def thermalSimulate(T, t, dx=0.1, A=1, T_ext=0, btype=(1,0,0,0,0,0)):
    """
    Simulates the diffusion of heat through a solid given an initial
    temperature distribution and boundary conditions. Note that a
    perfectly conducting boundary will not in fact result in an edge
    temperature exactly equal to the external temperature. The
    algorithm creates a surface just outside the given solid.
    
    Arguments:
    ---------
    - T: a 3D NumPy array containing the initial temperature distribution.
         
    - t: an array containing the simulation times; should have uniform separation
    
    - dx: the separation in cm between adjacent points in the solid.
        Default: 0.1 cm
    
    - A: the ratio K/(C*rho) where K is the conductivity, C is the specific
        heat capacity, and rho is the density.
        Default: 1
    
    - T_ext: Temperature of the external environment. This may be either
        a scalar or an array. If it is an array, then the external
        temperatures at the 6 surfaces may be specified separately.
        The surfaces must be listed in the order (x0, x1, y0, y1, z0, z1)
        where x0 is the surface with the first index equal to zero, x1 is the
        surface opposite x0, and likewise for the other 4 surfaces.
        Default: 0
        
    - btype: Array-like with the boundary types for the 6 surfaces.
        A value of 1 represents perfectly conducting and 0 represents
        perfectly insulating. Any value other than 0 or 1 will yield
        a linear combination of the two types.
        The surfaces are listed in the same order described above.
        Default: (1, 0, 0, 0, 0, 0)
    
    
    Returns:
    -------
    - simulation: a 4D NumPy array containing the results of the
         simulation, with the first three indices corresponding to
         the spatial coordinates, and the fourth index corresponding
         to the time
    
    """
    
    # Assume uniform times.
    dt = t[1]-t[0]
    
    
    # Modify the array to include fictional points
    currentT = np.vstack((T[[0],:,:], T, T[[-1],:,:]))
    currentT = np.hstack((currentT[:,[0],:], currentT, currentT[:,[-1],:]))
    currentT = np.dstack((currentT[:,:,[0]], currentT, currentT[:,:,[-1]]))
#     updateBoundary(currentT, T_ext, btype=btype)
    
    # Make an empty array with an extra axis for time
    simulation = np.zeros(T.shape + (len(t),))
    
    
    # First entry
    simulation[:,:,:,0] = T
    
    # Iterate!
    for i in range(1,len(t)):
        
        # Update the boundary, since we probably messed it up
        updateBoundary(currentT, T_ext, btype=btype)
        
        # Calculate the stuff as described in the project notebook
        T_sum = np.roll(currentT, -1, axis=0) + np.roll(currentT, 1, axis=0) \
              + np.roll(currentT, -1, axis=1) + np.roll(currentT, 1, axis=1) \
              + np.roll(currentT, -1, axis=2) + np.roll(currentT, 1, axis=2)
        deltaT = A*(T_sum - 6*currentT)/(dx**2) * dt
        
        # Update the temperature
        currentT = currentT + deltaT
        
        # Add to the simulation
        simulation[:,:,:,i] = currentT[1:-1,1:-1,1:-1]
    
    return simulation

### Step 3: Testing the Simulator ###

We carry out a simple test: if the solid begins at a temperature different from the heat bath, will the temperature equalize?

In [ ]:
# Initial temperature 0K
T = np.zeros((11, 11, 11))

# Time span of 2 seconds
t=np.linspace(0, 2, 2000)

# Simulate with an external temperature of 10 K
sim = thermalSimulate(T, t, T_ext=10)


In [ ]:
# Display a cross section from the middle of the solid
# observed at the end of the simulation
plt.imshow(sim[:,5,:,-1])
plt.title("Temperature in Cross-Section")
plt.figure()
# Plot the temperature from the insulated to the
# non-insulated end
plt.title("Temperature Graph")
plt.plot(sim[:,5,5,-1])
plt.ylim(0,10)
plt.xlim(0,10)
plt.xlabel("Depth")
plt.ylabel("Temperature")
# plt.gca().set_aspect('equal')
plt.figure()

For the next test, we will simulate the temperature flow in a conductor that is conductive at opposite ends, and insulated everywhere else. The conductive ends will be held at different temperatures. The final steady-state temperature of the solid should be linear.

In [ ]:
# Initial temperature 0K
T = np.zeros((11, 11, 11))

# Time span of 1 second
t=np.linspace(0, 1, 1000)

# Simulate with an external temperature of 1 K
sim = thermalSimulate(T, t, T_ext=(1, 0, 0, 0, 0, 0), btype=(1, 1, 0, 0, 0, 0))
# sim = thermalSimulate(T, t, T_ext=10)

# Display a cross section from the middle of the solid
# observed at the end of the simulation
plt.imshow(sim[:,5,:,-1])
plt.title("Temperature in Cross-Section")
plt.figure()
# Plot the temperature from the insulated to the
# non-insulated end
plt.title("Temperature Graph")
plt.plot(sim[:,5,5,-1])
plt.ylim(0,1)
plt.xlim(0,10)
plt.xlabel("Depth")
plt.ylabel("Temperature")
# plt.gca().set_aspect('equal')
plt.figure()

Now run the simulator using the values for a real substance. I'll use a cube of tin measuring 1 mm across. Since this is just a qualititive test, I'll grab the properties of tin from a quick Google search.

Specific heat capacity: 0.2177 J/(g$\cdot^{\circ}$C) [\[ref\]](https://www.engineersedge.com/materials/specific_heat_capacity_of_metals_13259.htm) \
Density: 5.769 g/cm$^3$  [\[ref\]](https://en.wikipedia.org/wiki/Tin) \
Thermal conductivity: 66.8 W/(m$\cdot$K)  [\[ref\]](https://en.wikipedia.org/wiki/Tin)

This gives CR = 1.256 J/(cm$^3 \cdot ^{\circ}$C).

In [ ]:
# Initial temperature 4K
T = np.ones((11, 11, 11))*4

# Parameters
K = 66.8 # Thermal conductivity
CR = 1.256 # Heat capacity times density
A = K/CR # Calculate the ratio
dx = 1e-4 # Cube side length


# Pick a more stable timestep
dt = 0.1*dx**2/A

t=np.arange(0, dt*1000, dt)

# Simulate with an external temperature of 10 K
# sim = thermalSimulate(T, t, T_ext=10, CR=1.256, K=66.8, dx=0.0001)

sim = thermalSimulate(T, t, T_ext=10, A=A, dx=dx)

In [ ]:
# Display a cross section from the middle of the solid
# observed at the end of the simulation
# plt.imshow(sim[:,5,:,-1])
# plt.figure()
# Plot the temperature from the insulated to the
# non-insulated end
plt.plot(sim[:,5,5,-1])
plt.ylim(0,10.1)
plt.xlim(0,10)
plt.figure()

### Step 4: Photon Absorption ###

With the simulator working, I now simulate a photon absorption event. I assume that an energetic photon is absorbed by one of the units in the solid, and then simulate the resulting heat dissipation. I include a function that produces a temperature/time plot for a particular location within the solid as the heat dissipates.

In [ ]:
K = 66.8 # Thermal conductivity
CR = 1.256 # Heat capacity times density
A = K/CR # Calculate the ratio
dx = 0.91e-4 # Cube side length


# Pick a more stable timestep
dt = 0.1*dx**2/A
t=np.arange(0, dt*3000, dt)

# Initial temperature 0K
T = np.ones((11, 11, 11))*0

# Energy of photon: 100 keV = 1.6e-14 J
E = 1.6e-14

# Temperature change from photon: energy divided by heat capacity
# of a single unit
deltaT = E/(CR * dx**3)

T[-1,5,5] = deltaT

# Simulate with an external temperature of 0 K
sim = thermalSimulate(T, t, A=A, T_ext=0, dx=dx)

In [ ]:
t_signature = sim[1,6,6,:]
plt.plot(t,t_signature)
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.title("100keV photon absorption event\nin 1 cubic mm of tin")

In [ ]:
def getSignature(p_loc, p_energy, dt=0.001, tmax=2.5, T_ext=0, sensor_loc=(1, 5, 5)):
    # Initial temperature
    T = np.ones((11, 11, 11))*T_ext
    T[p_loc[0], p_loc[1], p_loc[2]] = T_ext + p_energy

    t=np.arange(0, tmax, dt)

    sim = thermalSimulate(T, t, T_ext=T_ext)
    
    signature = sim[sensor_loc[0],sensor_loc[1],sensor_loc[2],:]
    
    return t, signature

def getSignature2(T, p_loc, p_energy=100., steps=1000, dx=0.1, A=1, T_ext=0., sensor_loc=(1, 5, 5)):
    dt = 0.1*dx**2/A
    t = np.arange(0, dt*steps, dt)
    
    # Initial temperature
    T = T*0.0 + T_ext
    T[p_loc[0], p_loc[1], p_loc[2]] = T_ext + p_energy

    sim = thermalSimulate(T, t, T_ext=T_ext)
    
    signature = sim[sensor_loc[0],sensor_loc[1],sensor_loc[2],:]
    
    return t, signature

In [ ]:
T = np.zeros((7,11,11))
signatures = [None]*4
labels = ("Center", "Corner", "5u Sideways", "5u Diagonal")
t, signatures[0] = getSignature2(T, (-1, 5, 5))
t, signatures[1] = getSignature2(T, (-1, 0, 0))
t, signatures[2] = getSignature2(T, (-1, 0, 5))
t, signatures[3] = getSignature2(T, (-1, 2, 1))


In [ ]:
plt.title("Photon strikes at different surface locations\non 7x11x11 absorber")
for i in range(len(signatures)):
    plt.plot(t, signatures[i], label=labels[i])
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

plt.title("Rescaled comparison of different strike locations")
# plt.plot(t, signatures[0], label="Center")
for i in range(0,len(signatures)):
    offset = t[np.argmax(signatures[i])]
    rescale = np.max(signatures[0])/np.max(signatures[i])
    plt.plot(t-offset, signatures[i]*rescale, label=labels[i])
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
T2 = np.zeros((11,11,11))
signatures = [None]*4
labels = ("Center", "Corner", "5u Sideways", "5u Diagonal")
t, signatures[0] = getSignature2(T2, (-1, 5, 5))
t, signatures[1] = getSignature2(T2, (-1, 0, 0))
t, signatures[2] = getSignature2(T2, (-1, 0, 5))
t, signatures[3] = getSignature2(T2, (-1, 2, 1))

In [ ]:
plt.title("Photon strikes at different surface locations\non 11x11x11 absorber")
for i in range(len(signatures)):
    plt.plot(t, signatures[i], label=labels[i])
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

plt.title("Rescaled comparison of different strike locations")
# plt.plot(t, signatures[0], label="Center")
for i in range(0,len(signatures)):
    offset = t[np.argmax(signatures[i])]
    rescale = np.max(signatures[0])/np.max(signatures[i])
    plt.plot(t-offset, signatures[i]*rescale, label=labels[i])
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
# Later I will make an absorber class to make the simulations easier
class absorber:
    def __init__(self, dim=(11,11,11), dx=0.1, A=1, dt=None):
        self.T = np.zeros(dim)
        

In [ ]:
t, sig = getSignature((10, 5, 5), 100)
plt.plot(t, sig)

In [ ]:

t, sig2 = getSignature((10, 0, 0), 100)
plt.plot(t, sig)
plt.plot(t, sig2)

In [ ]:

t, sig2 = getSignature((10, 0, 0), 100*np.max(sig)/np.max(sig2))

In [ ]:
shift = 70
plt.plot(t[:-shift], sig[:-shift])
plt.plot(t[:-shift], sig2[shift:])